# import libraries

In [13]:
import mlflow
import mlflow.sklearn
import pandas as pd
import os
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report,accuracy_score,confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt

# Read data

In [3]:
def read_file(*args,file_type="csv")->pd.DataFrame:
    """:parameter input directories in sequence
        :return DataFrame"""
    
    path=os.path.join(*args)
    if file_type == "tsv":
        return pd.read_csv(path, sep="\t")
    return pd.read_csv(path)

In [12]:
df=read_file("..","data","processed","cleaned.csv").dropna(how="any").drop_duplicates()

In [16]:
X= df["comment"]
y= df["category"]+1

In [21]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

In [22]:
y_train.unique()

array([0, 2, 1])

# Experiment-1 BoW vs TfIdf

In [19]:
mlflow.set_tracking_uri("http://localhost:5000")

In [20]:
mlflow.set_experiment("exp-1 BoW vs Tfldf")

2024/10/19 12:42:39 INFO mlflow.tracking.fluent: Experiment with name 'exp-1 BoW vs Tfldf' does not exist. Creating a new experiment.


<Experiment: artifact_location='/home/spynom/mlruns/12', creation_time=1729321959060, experiment_id='12', last_update_time=1729321959060, lifecycle_stage='active', name='exp-1 BoW vs Tfldf', tags={}>

In [23]:
def run_experiment(vectorizer_type,ngram_range,vectorizer_max_features,vectorizer_name):
    if vectorizer_type=="CountVectorizer":
        vectorizer=CountVectorizer(ngram_range=ngram_range,max_features=vectorizer_max_features,decode_error="ignore")
    else:
        vectorizer=TfidfVectorizer(ngram_range=ngram_range,max_features=vectorizer_max_features,decode_error="ignore")

    
    x_train=vectorizer.fit_transform(X_train)
    x_test=vectorizer.transform(X_test)
    
    with mlflow.start_run() as run:
        # set tags of experiment
        mlflow.set_tag("mlflow.runName",f"{vectorizer_name}_{ngram_range}_RandomForest")
        mlflow.set_tag("experiment_type","feature_extraction")
        mlflow.set_tag("model_type","RandomForestClassifier")
        
        # add a description
        mlflow.set_tag("description",f"RandomForestClassifier with {vectorizer_type}, ngram_range={ngram_range}")
        
        # Log vectorizer parameters
        mlflow.log_param("vectorizer_type", vectorizer_type)
        mlflow.log_param("ngram_range", ngram_range)
        mlflow.log_param("vectorizer_max_features", vectorizer_max_features)

        # Log Random Forest parameters
        n_estimators = 200
        max_depth = 15

        mlflow.log_param("n_estimators", n_estimators)
        mlflow.log_param("max_depth", max_depth)

        # Initialize and train the model
        model = RandomForestClassifier(n_estimators=n_estimators, max_depth=max_depth, random_state=42)
        model.fit(x_train, y_train)

        # Step 5: Make predictions and log metrics
        y_pred = model.predict(x_test)

        # Log accuracy
        accuracy = accuracy_score(y_test, y_pred)
        mlflow.log_metric("accuracy", accuracy)
        
         # Log classification report
        classification_rep = classification_report(y_test, y_pred, output_dict=True)
        for label, metrics in classification_rep.items():
            if isinstance(metrics, dict):
                for metric, value in metrics.items():
                    mlflow.log_metric(f"{label}_{metric}", value)

        # Log confusion matrix
        conf_matrix = confusion_matrix(y_test, y_pred)
        plt.figure(figsize=(8, 6))
        sns.heatmap(conf_matrix, annot=True, fmt="d", cmap="Blues")
        plt.xlabel("Predicted")
        plt.ylabel("Actual")
        plt.title(f"Confusion Matrix: {vectorizer_name}, {ngram_range}")
        plt.savefig("confusion_matrix.png")
        mlflow.log_artifact("confusion_matrix.png")
        plt.close()

        # Log the model
        mlflow.sklearn.log_model(model, f"random_forest_model_{vectorizer_name}_{ngram_range}")

In [24]:
# Run experiments for BoW and TF-IDF with different n-grams
ngram_ranges = [(1, 1), (1, 2), (1, 3)]  # unigrams, bigrams, trigrams
max_features = 5000  # Example max feature size

for ngram_range in ngram_ranges:
    # BoW Experiments
    run_experiment("BoW", ngram_range, max_features, vectorizer_name="BoW")

    # TF-IDF Experiments
    run_experiment("TF-IDF", ngram_range, max_features, vectorizer_name="TF-IDF")

2024/10/19 12:46:09 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
2024/10/19 12:46:09 INFO mlflow.tracking._tracking_service.client: 🏃 View run BoW_(1, 1)_RandomForest at: http://localhost:5000/#/experiments/12/runs/05ca343f154b4aadbb84bf010a5f2943.
2024/10/19 12:46:09 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: http://localhost:5000/#/experiments/12.
2024/10/19 12:46:16 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
2024/10/19 12:46:16 INFO mlflow.tracking._tracking_service.client: 🏃 View run TF-IDF_(1, 1)_RandomForest at: http://localhost:5000/#/experiments/12/runs/0baddf70463a46feaae71b7dd3fd7c95.
2024/10/19 12:46:16 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: http://localhost:50

# Experiment 2 tfIdf with different max_features

In [25]:
# Set or create an experiment
mlflow.set_experiment("TfIdf Trigram max_features")

2024/10/19 12:49:25 INFO mlflow.tracking.fluent: Experiment with name 'TfIdf Trigram max_features' does not exist. Creating a new experiment.


<Experiment: artifact_location='/home/spynom/mlruns/13', creation_time=1729322365460, experiment_id='13', last_update_time=1729322365460, lifecycle_stage='active', name='TfIdf Trigram max_features', tags={}>

In [27]:
# Function to run the experiment
def run_experiment_tfidf_max_features(max_features):
    ngram_range = (1, 3)  # Trigram setting

    #Vectorization using TF-IDF with varying max_features
    vectorizer = TfidfVectorizer(ngram_range=ngram_range, max_features=max_features)

    x_train=vectorizer.fit_transform(X_train)
    x_test=vectorizer.transform(X_test)

    # Define and train a Random Forest model
    with mlflow.start_run() as run:
        # Set tags for the experiment and run
        mlflow.set_tag("mlflow.runName", f"TFIDF_Trigrams_max_features_{max_features}")
        mlflow.set_tag("experiment_type", "feature_engineering")
        mlflow.set_tag("model_type", "RandomForestClassifier")

        # Add a description
        mlflow.set_tag("description", f"RandomForest with TF-IDF Trigrams, max_features={max_features}")

        # Log vectorizer parameters
        mlflow.log_param("vectorizer_type", "TF-IDF")
        mlflow.log_param("ngram_range", ngram_range)
        mlflow.log_param("vectorizer_max_features", max_features)

        # Log Random Forest parameters
        n_estimators = 200
        max_depth = 15

        mlflow.log_param("n_estimators", n_estimators)
        mlflow.log_param("max_depth", max_depth)

        # Initialize and train the model
        model = RandomForestClassifier(n_estimators=n_estimators, max_depth=max_depth, random_state=42)
        model.fit(x_train, y_train)

        # Step 5: Make predictions and log metrics
        y_pred = model.predict(x_test)

        # Log accuracy
        accuracy = accuracy_score(y_test, y_pred)
        mlflow.log_metric("accuracy", accuracy)

        # Log classification report
        classification_rep = classification_report(y_test, y_pred, output_dict=True)
        for label, metrics in classification_rep.items():
            if isinstance(metrics, dict):
                for metric, value in metrics.items():
                    mlflow.log_metric(f"{label}_{metric}", value)

        # Log confusion matrix
        conf_matrix = confusion_matrix(y_test, y_pred)
        plt.figure(figsize=(8, 6))
        sns.heatmap(conf_matrix, annot=True, fmt="d", cmap="Blues")
        plt.xlabel("Predicted")
        plt.ylabel("Actual")
        plt.title(f"Confusion Matrix: TF-IDF Trigrams, max_features={max_features}")
        plt.savefig("confusion_matrix.png")
        mlflow.log_artifact("confusion_matrix.png")
        plt.close()

        # Log the model
        mlflow.sklearn.log_model(model, f"random_forest_model_tfidf_trigrams_{max_features}")

# Step 6: Test various max_features values
max_features_values = [1000, 2000, 3000, 4000, 5000, 6000, 7000, 8000, 9000, 10000]

for max_features in max_features_values:
    run_experiment_tfidf_max_features(max_features)

2024/10/19 12:51:08 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
2024/10/19 12:51:08 INFO mlflow.tracking._tracking_service.client: 🏃 View run TFIDF_Trigrams_max_features_1000 at: http://localhost:5000/#/experiments/13/runs/2ab0b766b0f243a4a306eaacf22308ac.
2024/10/19 12:51:08 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: http://localhost:5000/#/experiments/13.
2024/10/19 12:51:20 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
2024/10/19 12:51:20 INFO mlflow.tracking._tracking_service.client: 🏃 View run TFIDF_Trigrams_max_features_2000 at: http://localhost:5000/#/experiments/13/runs/d89da4a694384f49b75dfaee30a6047e.
2024/10/19 12:51:20 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: http